## 一、层：深度学习的基础模块
神经网络的基本数据结构是层。层是一个数据处理模块，它接收一个或多个张量作为输入，并输出一个或多个张量。有些层是无状态的，但大多数层具有状态，即层的权重。权重是利用随机梯度下降学到的一个或多个张量，其中包含神经网络的知识（knowledge）。

不同类型的层适用于不同的张量格式和不同类型的数据处理。例如，简单的向量数据存储在形状为(samples, features)的2阶张量中，通常用密集连接层［densely connected layer，也叫全连接层（fully connected layer）或密集层（dense layer），对应于Keras的Dense类］来处理。序列数据存储在形状为(samples, timesteps, features)的3阶张量中，通常用循环层（recurrent layer）来处理，比如LSTM层或一维卷积层（Conv1D）。图像数据存储在4阶张量中，通常用二维卷积层（Conv2D）来处理。

我们可以把层看作深度学习的乐高积木，Keras将这个比喻具象化。在Keras中构建深度学习模型，就是将相互兼容的层拼接在一起，建立有用的数据变换流程。

### keras的Layer基类
简单的API应该具有单一的核心抽象概念。在Keras中，这个核心概念就是Layer类。Keras中的一切，要么是Layer，要么与Layer密切交互。

Layer是封装了状态（权重）和计算（一次前向传播）的对象。权重通常在build()中定义（不过也可以在构造函数__init__()中创建），计算则在call()方法中定义。在章节2.5中，我们实现了一个NaiveDense类，它包含两个权重W和b，并进行如下计算：output =activation(dot(input, W) + b)。Keras的层与之非常相似.

#### Dense层的实现：作为Layer的子类

In [5]:
from tensorflow import keras
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore") # 忽略警告信息，不影响代码执行

class SimpleDense(keras.layers.Layer): # Keras的所有层都继承自Layer基类
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):  # 在build()方法中创建权重
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units),  # add_weight()是创建权重的快捷方法。你也可以创建独立变量，并指定其作为层属性，比如self.W = tf.Variable(tf.random.uniform(w_shape))
                                 initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros")

    def call(self, inputs):  # 在call()方法中定义前向传播计算
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [7]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu) # 将上面定义的层实例化
input_tensor = tf.ones(shape=(2, 784)) # 创建一些测试输入
output_tensor = my_dense(input_tensor) # 对输入调用层，就像调用函数一样
print(output_tensor.shape)

(2, 32)


### 自动推断形状：动态构建层
就像玩乐高积木一样，你只能将兼容的层“拼接”在一起。层兼容性（layer compatibility）的概念具体指的是，每一层只接收特定形状的输入张量，并返回特定形状的输出张量。

In [8]:
from tensorflow.keras import layers
layer = layers.Dense(32, activation="relu") # 有32个输出单元的密集层

该层将返回一个张量，其第一维的大小已被转换为32。它后面只能连接一个接收32维向量作为输入的层。

在使用Keras时，往往不必担心尺寸兼容性问题，因为添加到模型中的层是动态构建的，以匹配输入层的形状，例如下述代码：

In [9]:
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(32)
])

这些层没有收到任何关于输入形状的信息；相反，它们可以自动推断，遇到第一个输入的形状就是其输入形状。

在前面2.5节实现的简单Dense层中（我们将其命名为NaiveDense），我们必须将该层的输入大小明确传递给构造函数，以便能够创建其权重。这种方法并不理想，因为它会导致模型的每个新层都需要知道前一层的形状。

``
model = NaiveSequential([
    NaiveDense(input_size=784, output_size=32, activation="relu"),
    NaiveDense(input_size=32, output_size=64, activation="relu"),
    NaiveDense(input_size=64, output_size=32, activation="relu"),
    NaiveDense(input_size=32, output_size=10, activation="softmax")
])model = NaiveSequential([
    NaiveDense(input_size=784, output_size=32, activation="relu"),
    NaiveDense(input_size=32, output_size=64, activation="relu"),
    NaiveDense(input_size=64, output_size=32, activation="relu"),
    NaiveDense(input_size=32, output_size=10, activation="softmax")
])
``

有了自动形状推断，前面的示例就变得简洁了，如下所示

In [11]:
model = keras.Sequential([
    SimpleDense(32, activation="relu"),
    SimpleDense(64, activation="relu"),
    SimpleDense(32, activation="relu"),
    SimpleDense(10, activation="softmax")
]
)

## 二、从层到模型

深度学习模型是由层构成的图，在Keras中就是Model类。到目前为止，我们只见过Sequential模型（Model的一个子类），它是层的简单堆叠，将单一输入映射为单一输出。但随着深入学习，我们会接触到更多类型的网络拓扑结构。一些常见的结构包括：双分支（two-branch）网络、多头（multihead）网络、残差连接，网络拓扑结构可能会非常复杂。

在Keras中构建模型通常有两种方法：直接作为Model类的子类，或者使用函数式API，后者可以用更少的代码做更多的事情。

模型的拓扑结构定义了一个**假设空间**（机器学习就是在预先定义的可能性空间内，利用反馈信号的指引，寻找特定输入数据的有用表示）。通过选择网络拓扑结构，我们可以将可能性空间（假设空间）限定为一系列特定的张量运算，将输入数据映射为输出数据。然后，我们要为这些张量运算的权重张量寻找一组合适的值。

要从数据中学习，我们必须对其进行假设。这些假设定义了可学习的内容。因此，假设空间的结构（模型架构）是非常重要的。它编码了我们对问题所做的假设，即模型的先验知识。如果我们正在处理一个二分类问题，使用的模型由一个没有激活的Dense层组成（纯仿射变换），那么我们就是在假设这两个类别是线性可分的。

选择正确的网络架构，更像是一门艺术而不是科学。虽然有一些最佳实践和原则，但只有实践才能帮助我们成为合格的神经网络架构师。
**每种类型的模型架构适合解决哪类问题？在实践中如何构建这些网络？如何选择正确的学习配置？如何调节模型，直到产生我们想要的结果？**

## 三、编译步骤：配置学习过程
一旦确定了模型架构，我们还需要选定以下3个参数。
* **损失函数（目标函数）**——在训练过程中需要将其最小化。它衡量的是当前任务是否成功。
* **优化器**——决定如何基于损失函数对神经网络进行更新。它执行的是随机梯度下降（SGD）的某个变体。
* **指标**——衡量成功的标准，在训练和验证过程中需要对其进行监控，如分类精度。与损失不同，训练不会直接对这些指标进行优化。因此，指标不需要是可微的。

一旦选定了损失函数、优化器和指标，就可以使用内置方法compile()和fit()开始训练模型。此外，也可以编写自定义的训练循环。

### compile( )
compile()方法的作用是配置训练过程，它接收的参数是optimizer、loss和metrics（一个列表）

In [14]:
model = keras.Sequential([keras.layers.Dense(1)]) # 定义一个线性分类器
model.compile(
    optimizer="rmsprop", # 指定优化器的名称：RMSprop（不区分大小写）
    loss="mean_squared_error", # 制定损失函数的名称：均方误差
    metrics=["accuracy"] # 指定指标列表：本例中只有精度
)

在上面对compile()的调用中，我们把优化器、损失函数和指标作为字符串（如"rmsprop"）来传递。这些字符串实际上是访问Python对象的快捷方式。例如，"rmsprop"会变成keras.optimizers.RMSprop()。重要的是，也可以把这些参数指定为对象实例，如下所示

In [15]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.MeanAbsoluteError(),
              metrics=[keras.metrics.BinaryAccuracy()])

如果我们想传递自定义的损失函数或指标，或者想进一步配置正在使用的对象，比如向优化器传入参数learning_rate

``
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
             loss=my_custom_loss,
             metrics=[my_custom_metric1, my_custom_metric2])           
``

一般来说，我们无须从头开始创建自己的损失函数、指标或优化器，因为Keras提供了下列多种内置选项，很可能满足你的需求。
#### 优化器
* SGD（带动量或不带动量）
* RMSprop
* Adam
* Adagrad
* 等等

#### 损失函数：
* CategoricalCrossentropy
* SparseCategoricalCrossentropy
* BinaryCrossentropy
* MeanSquaredError
* KLDivergence
* CosineSimilarity
* 等等

#### 指标
* CategoricalAccuracy
* SparseCategoricalAccuracy
* BinaryAccuracy
* AUC
* Precision
* Recall
* 等等

## 四、选择损失函数
为问题选择合适的损失函数，这是极其重要的。神经网络会采取各种方法使损失最小化，如果损失函数与成功完成当前任务不完全相关，那么神经网络最终的结果可能会不符合你的预期。

比如对于分类、回归和序列预测等常见问题，可以遵循一些简单的指导原则来选择合适的损失函数。例如，**对于二分类问题，可以使用二元交叉熵损失函数；对于多分类问题，可以使用分类交叉熵损失函数。只有在面对全新的研究问题时，你才需要自己开发损失函数。**

## 五、理解fit（）方法
compile()之后将是fit()。fit()方法执行训练循环，它有以下关键参数。
* 要训练的**数据**（输入和目标）：这些数据通常以NumPy数组或TensorFlow Dataset对象的形式传入。
* 训练**轮数**：训练循环应该在传入的数据上迭代多少次。
* 在每轮小批量梯度下降中使用的**批量大小**：在一次权重更新中，计算梯度所要考虑的训练样本的数量。

#### 对NumPy数据调用fit()

In [22]:
# 预制信息（上一章节内容）
import numpy as np
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal( 
    mean=[0, 3],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class) #  生成第一个类别的点：1000个二维随机点。协方差矩阵为[[1, 0.5], [0.5, 1]]，对应于一个从左下方到右上方的椭圆形点云

positive_samples = np.random.multivariate_normal(  
    mean=[3, 0],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class) # 生成第二个类别的点，协方差矩阵相同，均值不同

inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype='float32'), np.ones((num_samples_per_class, 1), dtype='float32')))


history = model.fit(
    inputs, # 输入样本，一个Numpy数组
    targets, # 对应的训练目标，一个Numpy数组
    epochs=5, # 训练循环对数据迭代5次
    batch_size=128 # 训练循环的批量大小为128
)
history.history

Epoch 1/5
16/16 [==============================] - 0s 2ms/step - loss: 3.5149 - binary_accuracy: 0.4880
Epoch 2/5
16/16 [==============================] - 0s 2ms/step - loss: 3.5088 - binary_accuracy: 0.4890
Epoch 3/5
16/16 [==============================] - 0s 2ms/step - loss: 3.5027 - binary_accuracy: 0.4890
Epoch 4/5
16/16 [==============================] - 0s 2ms/step - loss: 3.4965 - binary_accuracy: 0.4890
Epoch 5/5
16/16 [==============================] - 0s 1ms/step - loss: 3.4904 - binary_accuracy: 0.4890


{'loss': [3.514859676361084,
  3.5088045597076416,
  3.502668619155884,
  3.4965391159057617,
  3.4904370307922363],
 'binary_accuracy': [0.4880000054836273,
  0.48899999260902405,
  0.48899999260902405,
  0.48899999260902405,
  0.48899999260902405]}

## 六、监控验证数据上的损失和指标
机器学习的目标不是得到一个在训练数据上表现良好的模型——做到这一点很容易，只需跟随梯度下降即可。机器学习的目标是得到总体上表现良好的模型，特别是在模型未见过的数据上。一个模型在训练数据上表现良好，并不意味着它在未见过的数据上也会表现良好。举例来说，模型有可能只是记住了训练样本和目标值之间的映射关系，但这对在未见过的数据上进行预测毫无用处。

要想查看模型在新数据上的性能，标准做法是保留训练数据的一个子集作为验证数据（validation data）。我们不会在这部分数据上训练模型，但会用它来计算损失值和指标值。实现方法是在fit()中使用validation_data参数。和训练数据一样，验证数据也可以作为NumPy数组或TensorFlow Dataset对象传入。

#### 使用validation_data参数

In [25]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
             loss=keras.losses.MeanAbsoluteError(),
             metrics=[keras.metrics.BinaryAccuracy()]
             )

#为避免验证数据都来自同一个类别，需要使用随机索引排列将数据打乱
indices_permutation = np.random.permutation(len(inputs))  
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

# 保留30%的训练数据用于验证（我们不会将这部分数据用于训练，而是保留下来用于计算验证损失和指标）
num_validation_samples = int(0.3 * len(inputs))  
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]
model.fit(
    training_inputs,  # 训练数据，用于更新模型权重
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets) # 验证数据，仅用来监控验证损失和指标
)

Epoch 1/5
88/88 [==============================] - 1s 3ms/step - loss: 0.3185 - binary_accuracy: 0.9421 - val_loss: 0.1511 - val_binary_accuracy: 1.0000
Epoch 2/5
88/88 [==============================] - 0s 1ms/step - loss: 0.2230 - binary_accuracy: 0.9579 - val_loss: 0.2010 - val_binary_accuracy: 0.9917
Epoch 3/5
88/88 [==============================] - 0s 1ms/step - loss: 0.2237 - binary_accuracy: 0.9471 - val_loss: 0.2455 - val_binary_accuracy: 0.9317
Epoch 4/5
88/88 [==============================] - 0s 1ms/step - loss: 0.2145 - binary_accuracy: 0.9593 - val_loss: 0.1692 - val_binary_accuracy: 1.0000
Epoch 5/5
88/88 [==============================] - 0s 1ms/step - loss: 0.2216 - binary_accuracy: 0.9621 - val_loss: 0.2414 - val_binary_accuracy: 0.9617


在验证数据上的损失值叫作“验证损失”，以区别于“训练损失”。请注意，必须将训练数据和验证数据严格分开：验证的目的是监控模型所学到的知识在新数据上是否真的有用。如果验证数据在训练期间被模型看到过，那么验证损失和指标就会不准确。

In [27]:
# 调用evaluate()方法,在训练完成后计算验证损失和指标
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)
loss_and_metrics

5/5 [==============================] - 0s 1ms/step - loss: 0.2414 - binary_accuracy: 0.9617


[0.2413594126701355, 0.9616666436195374]

## 七、推断：在训练后使用模型
一旦训练好了模型，就可以用它来对新的数据进行预测。这叫作推断（inference）。要做到这一点，一个简单的方法就是调用该模型（__call__()）。

``
predictions = model(new_inputs)  # 接收一个NumPy数组或TensorFlow张量，返回一个TensorFlow张量
``

但是，这种方法会一次性处理new_inputs中的所有输入，如果其中包含大量数据，那么这种方法可能是不可行的（尤其是，它可能需要比你的GPU更大的内存）。要想进行推断，一种更好的方法是使用predict()方法。它将小批量地迭代数据，并返回预测值组成的NumPy数组。与__call__()不同，它还可以处理TensorFlow Dataset对象。

``
predictions = model.predict(new_inputs, batch_size=128)  # 接收一个NumPy数组或Dataset对象，返回一个NumPy数组
``

In [28]:
# 对于前面训练的线性模型，如果对一些验证数据使用predict()，那么我们会得到一些标量值，对应于模型对每个输入样本的预测结果
predictions = model.predict(val_inputs, batch_size=128)
print(predictions[:10])

5/5 [==============================] - 0s 1ms/step
[[-0.15364945]
 [-0.5753247 ]
 [-0.06858563]
 [-0.17877996]
 [ 0.7428176 ]
 [ 0.6336271 ]
 [ 0.14723396]
 [ 0.7869841 ]
 [-0.03267717]
 [ 0.7790372 ]]
